In [ ]:
!pip3 install karateclub
import networkx as nx
from karateclub import BigClam
from random import randint

  Using cached https://files.pythonhosted.org/packages/5d/ba/d4dd40bc3cf8084d463ebb7c368c1efcb465b8b6d7586e7988482915189b/karateclub-1.0.14.tar.gz
  Using cached https://files.pythonhosted.org/packages/22/e7/4b2bdddb99f5f631d8c1de259897c2b7d65dcfcc1e0a6fd17a7f62923500/numpy-1.19.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9b/cd/dc52755d30ba41c60243235460961fc28022e5b6731f16c268667625baea/networkx-2.5-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/7e/281edb5bc3274dfb894d90f4dbacfceaca381c2435ec6187a2c6f329aed7/tqdm-4.48.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/31/d4/d244fa4ca96af747b9204ce500e8919ce72eab60de2129b22a45434f1598/python-louvain-0.14.tar.gz
    40% |█████████████                   | 2.8MB 81kB/s eta 0:00:509

In [3]:
def clique_hub(g, size, idx):
  for i in range(idx, idx+size):
    for j in range(i+1, idx+size):
      g.add_edge(i,j)
    g.add_edge(i,0)

In [ ]:
def clique(g,size,idx):
    for i in range(idx, idx+size):
        for j in range(i+1, idx+size):
            g.add_edge(i,j)

In [ ]:
def organize(member):
    # print(member)
    final = {}
    for x in member.keys():
        final[member[x]] = []
    for x in member.keys():
        final[member[x]].append(x)
    cnt = 1
    for x in final.keys():
        print("Community",str(cnt)+":",final[x])
        cnt = cnt + 1

In [ ]:
def hub_node(clq):
    #Cliques of size 1 to clq with each connected to a common node
    g = nx.Graph()
    num_nodes = int((clq*(clq+1))/2) + 1
    for i in range(num_nodes):
        g.add_node(i) 
    size = 1
    for i in range(1,clq+1):
        clique_hub(g,i,size)
        size = size + i
    model = BigClam()
    model.fit(g)
    cluster_membership = model.get_memberships()
    organize(cluster_membership)
    nx.draw(g, with_labels=True)

In [ ]:
def alt_hub_node(clq):
    #Cliques of size 1 to clq (no even number cliques) with each connected to a common node
    if(clq%2 == 0):
        clq = clq + 1
    g = nx.Graph()
    num_nodes = (clq+1)*(clq+1)//4 + 1
    for i in range(num_nodes):
        g.add_node(i)
    size = 1
    for i in range(1,clq+1, 2):
        clique_hub(g,i,size)
        size = size + i
    model = BigClam()
    model.fit(g)
    cluster_membership = model.get_memberships()
    organize(cluster_membership)
    nx.draw(g, with_labels=True)

In [ ]:
def hub_clq_clq(clq1, clq2):
    # 2 cliques connected by a hub node which is connected to all nodes of the graph
    g = nx.Graph()
    num_nodes = 1 + clq1 + clq2
    for i in range(num_nodes):
        g.add_node(i)
    clique_hub(g,clq1,1)
    clique_hub(g,clq2,1+clq1)
    model = BigClam()
    model.fit(g)
    cluster_membership = model.get_memberships()
    organize(cluster_membership)
    nx.draw(g, with_labels=True)

In [ ]:
def node_clq_clq(clq1, clq2):
    # 2 cliques connected by a hub node which is connected to all nodes of the graph
    g = nx.Graph()
    num_nodes = 1 + clq1 + clq2
    for i in range(num_nodes):
        g.add_node(i)
    clique_hub(g,clq1,1)
    clique_hub(g,clq2,1+clq1)
    for i in range(2, num_nodes):
        if(i == 1 + clq1):
            continue
        g.remove_edge(0,i)
    model = BigClam()
    model.fit(g)
    cluster_membership = model.get_memberships()
    organize(cluster_membership)
    nx.draw(g, with_labels=True)

In [ ]:
def big_small(big_num, small_num, num):
    # A big community of size big_num surrounded by num small communities of size small_num
    g = nx.Graph()
    num_nodes = big_num + small_num*num
    for i in range(num_nodes):
        g.add_node(i)
    clique_hub(g,big_num,0)
    for i in range(num):
        clique(g,small_num,big_num + i*small_num)
        g.add_edge(randint(0,big_num-1),big_num + i*small_num)
    model = BigClam()
    model.fit(g)
    cluster_membership = model.get_memberships()
    organize(cluster_membership)
    nx.draw(g, with_labels=True)

In [ ]:
def ring_cliques(arr):
    # A ring of cliques with len(arr) clqieus and each clique i being is size arr[i]
    g = nx.Graph()
    if(len(arr) == 0):
        return
    num_nodes = 0
    for x in arr:
        num_nodes = num_nodes + x
    size = arr[0]
    clique(g,arr[0],0)
    for i in range(1,len(arr)):
        clique(g, arr[i], size)
        g.add_edge(size,size - 1)
        size = size + arr[i]
    g.add_edge(0,num_nodes-1)
    model = BigClam()
    model.fit(g)
    cluster_membership = model.get_memberships()
    organize(cluster_membership)
    nx.draw(g, with_labels=True)   

In [ ]:
hub_node(10)

In [ ]:
alt_hub_node(11)

In [ ]:
hub_clq_clq(7,10)

In [ ]:
node_clq_clq(7,10)

In [ ]:
big_small(100,5,5)

In [ ]:
big_small(10, 30, 5)

In [ ]:
ring_cliques([15,8,10,7,5,10])